In [1]:
from dataset.dataset import Dataset, load_image

import numpy as np
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch
from Net.resnext import resnext101_32x8d_wsl
import torchvision.models as models
from Net.fast_rcnn import FasterRCNNVGG16
from utils.get_layer import IntermediateLayerGetter
from Net.roi_pool import VGG16RoIHead

In [2]:
# %%
carpath = "nuimages/train/car.json"
humanpath = "nuimages/train/human.json"
car_imgs = []
car_imgs = load_image(carpath, 0)


print(car_imgs[2]['annotations'])


[{'token': '000091dc8fbc4079b412f735aa743c71', 'category_token': 'fd69059b62a3469fbaef25340c0eab7f', 'bbox': [0, 484, 558, 893], 'mask': {'size': [900, 1600], 'counts': 'aWEwVTptYTAxTzJOMk4yW09kMDAxTzEwME8xTzAwMU8xMDAwTzEwME8xMDAwME8xMDBPMTAwTzEwME8xMDAwME8xMDBPMTAwTzJPMU4xMDFPMU4yTzBPMk9PMDEwTzFPMDAxME8wMU8wMTBPMU8wMDEwTzAxTzAwMTAwTzAwMTBPMDEwME8wMDEwTzAxME8wMTAwTzAxME9OMk00TDNNNEwzTTRMM000TDNNM000TDNNNEwzTTRMM04zTTJNNE0yTjJPMk0yTzJOMU4zTjFGO0Y5MDAwMU8wMDFPME8yTzAwMU8wMDAwMU8wMDFPMDAxMDBPMU8xTzFPMU8wMDFPMTAwTzFPMU8xTzFPMDAxTzEwME8xTzFPMU8wMDFPMU8xMDBPMU8xTzAwMU8xTzFPMDEwTzFPMU8wMDFPMU8xTzAxME8xTzFPMU8wMDFPMU8xME8wMU8xTzFPMDAxMDBPMU8wMDEwME8xTzAxME8xTzFPMTAwTzFPMU8xMDBPMU8xTzEwME8xTzEwME8xTzFPMTAwTzFPMU8xMDBPMU8xMDBPMU8xTzEwME8xTzEwME8xTzEwME8xTzEwMU4xMDBPMU8xMDBPMU8xMDBPMTAwTzFPMTAwTzEwME8xTzEwME8xTzEwME8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMU8xMDBPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzFPMTAwTzEwME8xTzEwME8xTzEwME8xTzEwME8xTzEwME

In [3]:
car_train = Dataset(car_imgs)
car_train.__getitem__(1)

(array([[[-1.8730536 , -1.8707865 , -2.0483842 , ..., -2.009823  ,
          -2.0535338 , -2.0731025 ],
         [-1.79644   , -1.3509831 , -1.8635465 , ..., -1.6560079 ,
          -1.7277278 , -1.6506724 ],
         [-1.9638543 , -1.4887582 , -2.0664687 , ..., -1.732536  ,
          -1.7522357 , -1.4234526 ],
         ...,
         [-1.7192507 , -1.2069919 , -1.2110233 , ..., -1.3464978 ,
          -1.4784987 , -1.3929509 ],
         [-1.7117914 , -1.0935878 , -1.2236992 , ..., -1.4126201 ,
          -1.2633728 , -1.0973418 ],
         [-1.8630438 , -1.3130984 , -1.4610538 , ..., -1.154789  ,
          -0.98831916, -1.0472839 ]],
 
        [[-1.9625369 , -1.8667592 , -1.8507848 , ..., -1.9243641 ,
          -1.8776215 , -1.7259119 ],
         [-1.7860494 , -1.1772574 , -1.4979502 , ..., -1.8021265 ,
          -1.7787195 , -1.6543077 ],
         [-1.695717  , -1.1975026 , -1.6377726 , ..., -1.9214203 ,
          -1.9100968 , -1.648311  ],
         ...,
         [-1.7942672 , -1.1439506

In [4]:
batch_size = 2
car_train_data =  DataLoader(dataset=car_train, batch_size=batch_size, shuffle=True, num_workers=0)
Cuda = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
fast_rcnn_model = FasterRCNNVGG16().cuda()

C:\Users\Jiarun\AppData\Local\anaconda3\envs\ECE449project\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Jiarun\AppData\Local\anaconda3\envs\ECE449project\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
rpn_locs, rpn_scores, rois, roi_indices, anchor = rpn_model(conv5_3, (32,56))
# print(roi_indices)
roi_cls_locs, roi_scores = ROI_head_model(conv5_3,rois,roi_indices)
# print(roi_cls_locs)
print(roi_scores)

In [ ]:
backbone['classifier'][:-3].eval()